
<a id='fundamental-types'></a>
How to read this lecture…

- For some notebooks, enable content with “Trust” on the command tab of Jupyter lab  
- Code should execute sequentially if run in a Jupyter notebook  
- Please direct feedback to [contact@quantecon.org](mailto:contact@quantecon.org") or [discourse forum](http://discourse.quantecon.org/)  

# Contents

- [Overview](#Overview)  
- [Array Basics](#Array-Basics)  
  - [Shape and Dimension](#Shape-and-Dimension)  
  - [Creating Arrays](#Creating-Arrays)  
  - [Array Indexing](#Array-Indexing)  
  - [Views and Slices](#Views-and-Slices)  
  - [Special Matrices](#Special-Matrices)  
  - [Assignment and Passing Arrays](#Assignment-and-Passing-Arrays)  
- [Operations on Arrays](#Operations-on-Arrays)  
  - [Array Methods](#Array-Methods)  
  - [Matrix Algebra](#Matrix-Algebra)  
  - [Elementwise Operations](#Elementwise-Operations)  
  - [Broadcasting Functions](#Broadcasting-Functions)  
- [Introduction to Types](#Introduction-to-Types)  
  - [Good Practices for Functions and Variables](#Good-Practices-for-Functions-and-Variables)  
  - [Manually Declaring Types](#Manually-Declaring-Types)  
- [Linear Algebra](#Linear-Algebra)  
- [Ranges](#Ranges)  
- [Tuples and Named Tuples](#Tuples-and-Named-Tuples)  
- [Exercises](#Exercises)  
  - [Exercise 1](#Exercise-1)  
- [Solutions](#Solutions)  
  - [Exercise 1](#Exercise-1)  

> “Let’s be clear: the work of science has nothing whatever to do with consensus.
> Consensus is the business of politics. Science, on the contrary, requires only
> one investigator who happens to be right, which means that he or she has
> results that are verifiable by reference to the real world. In science
> consensus is irrelevant. What is relevant is reproducible results.” – Michael Crichton

# Overview

In Julia, arrays and tuples are the most important data type for working with numerical data

In this lecture we give more details on

- introduction to types  
- creating and manipulating Julia arrays  
- fundamental array processing operations  
- basic matrix algebra  
- tuples and named tuples  
- ranges  

# Array Basics

## Shape and Dimension

Activate the project environment, ensuring that `Project.toml` and `Manifest.toml` are in the same location as your notebook

In [1]:
using Pkg; Pkg.activate(@__DIR__); #activate environment in the notebook's location
using LinearAlgebra, Statistics

We’ve already seen some Julia arrays in action

In [2]:
a = [10, 20, 30]

3-element Array{Int64,1}:
 10
 20
 30

In [3]:
a = [1.0, 2.0, 3.0]

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

The REPL tells us that the arrays are of types `Array{Int64,1}` and `Array{Float64,1}` respectively

Here `Int64` and `Float64` are types for the elements inferred by the compiler

We’ll talk more about types later on

The `1` in `Array{Int64,1}` and `Array{Any,1}` indicates that the array is
one dimensional (i.e., a `Vector`)

This is the default for many Julia functions that create arrays

In [4]:
typeof(randn(100))

Array{Float64,1}

In Julia, one dimensional vectors are best interpreted as column vectors, which we will see when we take transposes

We can check the dimensions of `a` using `size()` and `ndims()`
functions

In [5]:
ndims(a)

1

In [6]:
size(a)

(3,)

The syntax `(3,)` displays a tuple containing one element — the size along the one dimension that exists

### Array vs Vector vs Matrix

In Julia, `Vector` and `Matrix` are just aliases for one- and two-dimensional arrays
respectively

In [7]:
Array{Int64, 1} == Vector{Int64}
Array{Int64, 2} == Matrix{Int64}

true

Vector construction with `,` is then interpreted as a column vector

To see this, we can create a column vector and row vector more directly

In [8]:
[1, 2, 3] == [1; 2; 3] #both column vectors

true

In [9]:
[1 2 3] #a row vector is 2-dimensional

1×3 Array{Int64,2}:
 1  2  3

As we’ve seen, in Julia we have both

- one-dimensional arrays (i.e., flat arrays)  
- arrays of size `(1, n)` or `(n, 1)` that represent row and column vectors respectively  


Why do we need both?

On one hand, dimension matters when we come to matrix algebra

- Multiplying by a row vector is different to multiplication by a column vector  


On the other, we use arrays in many settings that don’t involve matrix algebra

In such cases, we don’t care about the distinction between row and column vectors

This is why many Julia functions return flat arrays by default


<a id='creating-arrays'></a>

## Creating Arrays

### Functions that Create Arrays

We’ve already seen some functions for creating a vector filled with `0.0`

In [10]:
zeros(3)

3-element Array{Float64,1}:
 0.0
 0.0
 0.0

This generalizes to matrices and higher dimensional arrays

In [11]:
zeros(2, 2)

2×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0

To return an array filled with a single value, use `fill`

In [12]:
fill(5.0, 2, 2)

2×2 Array{Float64,2}:
 5.0  5.0
 5.0  5.0

Finally, you can create an empty array using the `Array()` constructor

In [13]:
x = Array{Float64}(undef, 2, 2)

2×2 Array{Float64,2}:
 6.91832e-310  6.91832e-310
 6.91832e-310  6.91832e-310

The printed values you see here are just garbage values

(the existing contents of the allocated memory slots being interpreted as 64 bit floats)

If you need more control over the types, fill with a non-floating point

In [14]:
fill(0, 2, 2) # fills with 0, not 0.0

2×2 Array{Int64,2}:
 0  0
 0  0

Or fill with a boolean type

In [15]:
fill(false, 2, 2) # produces a boolean matrix

2×2 Array{Bool,2}:
 false  false
 false  false

### Creating Arrays from Existing Arrays

For the most part, we will avoid directly specifying the types of arrays, and let the compiler deduce the optimal types on its own

The reasons for this, discussed in more detail in generic and functional programming, are to ensure both clarity and generality

One place this can be inconvenient is when we need to create an array based on an existing array

First, note that assignment in Julia binds a name to a value, but does not make a copy of that type

In [16]:
x = [1, 2, 3]
y = x
y[1] = 2
x

3-element Array{Int64,1}:
 2
 2
 3

In the above, the `y = x` simply create a new named binding called `y` which refers to whatever `x` currently binds to

To copy the data, you need to be more explicit

In [17]:
x = [1, 2, 3]
y = copy(x)
y[1] = 2
x

3-element Array{Int64,1}:
 1
 2
 3

However, rather than making a copy of `x`, you may want to just have a similarly sized array

In [18]:
x = [1, 2, 3]
y = similar(x)
y

3-element Array{Int64,1}:
 140028344648824
 140028343967520
 140028343969536

Similar can also be used to pre-allocate a vector with a different size, but the same shape

In [19]:
x = [1, 2, 3]
y = similar(x, 4) # make a vector of length 4

4-element Array{Int64,1}:
 140028504193472
 140028343882816
 140028343882560
 140028347156464

Which generalized to higher dimensions

In [20]:
x = [1, 2, 3]
y = similar(x, 2, 2) # make 2x2 matrix

2×2 Array{Int64,2}:
 140028445811904  140028347154672
 140028367743616                0

### Manual Array Definitions

As we’ve seen, you can create one dimensional arrays from manually specified data like so

In [21]:
a = [10, 20, 30, 40]

4-element Array{Int64,1}:
 10
 20
 30
 40

In two dimensions we can proceed as follows

In [22]:
a = [10 20 30 40]  # two dimensional, shape is 1 x n

1×4 Array{Int64,2}:
 10  20  30  40

In [23]:
ndims(a)

2

In [24]:
a = [10 20; 30 40]  # 2 x 2

2×2 Array{Int64,2}:
 10  20
 30  40

You might then assume that `a = [10; 20; 30; 40]` creates a two dimensional column vector but this isn’t the case

In [25]:
a = [10; 20; 30; 40]

4-element Array{Int64,1}:
 10
 20
 30
 40

In [26]:
ndims(a)

1

Instead transpose the matrix (or adjoint if complex)

In [27]:
a = [10 20 30 40]'

4×1 Adjoint{Int64,Array{Int64,2}}:
 10
 20
 30
 40

In [28]:
ndims(a)

2

## Array Indexing

We’ve already seen the basics of array indexing

In [29]:
a = [10 20 30 40]
a[end-1]

30

In [30]:
a[1:3]

3-element Array{Int64,1}:
 10
 20
 30

For 2D arrays the index syntax is straightforward

In [31]:
a = randn(2, 2)
a[1, 1]

-0.8010702218755774

In [32]:
a[1, :]  # First row

2-element Array{Float64,1}:
 -0.8010702218755774
  0.543123909816551 

In [33]:
a[:, 1]  # First column

2-element Array{Float64,1}:
 -0.8010702218755774
 -0.8802890632105388

Booleans can be used to extract elements

In [34]:
a = randn(2, 2)

2×2 Array{Float64,2}:
  1.3025    1.25139 
 -0.397159  0.628097

In [35]:
b = [true false; false true]

2×2 Array{Bool,2}:
  true  false
 false   true

In [36]:
a[b]

2-element Array{Float64,1}:
 1.3025031177386786
 0.6280970908515128

This is useful for conditional extraction, as we’ll see below

An aside: some or all elements of an array can be set equal to one number using slice notation

In [37]:
a = zeros(4)

4-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0

In [38]:
a[2:end] .= 42

3-element view(::Array{Float64,1}, 2:4) with eltype Float64:
 42.0
 42.0
 42.0

In [39]:
a

4-element Array{Float64,1}:
  0.0
 42.0
 42.0
 42.0

## Views and Slices

Using the `:` notation provides a slice of an array, copying the sub-array to a new array with a similar type

In [40]:
a = [1 2; 3 4]
b = a[:, 2]
@show b
a[:, 2] = [4, 5] # modify a
@show a
@show b;

b = [2, 4]
a = [1 4; 3 5]
b = [2, 4]


A `view` on the other hand does not copy the value

In [41]:
a = [1 2; 3 4]
@views b = a[:, 2]
@show b
a[:, 2] = [4, 5]
@show a
@show b;

b = [2, 4]
a = [1 4; 3 5]
b = [4, 5]


Note that the only difference is the `@views` macro, which will replace any slices with views in the expression

An alternative is to call the `view` function directly–though it is generally discouraged since it is a step away from the math

In [42]:
@views b = a[:, 2]
view(a, :, 2) == b

true

As with most programming in Julia, it is best to avoid prematurely assuming that `@views` will have a significant impact on performance, and stress code clarity above all else

Another important lesson about views is that they **are not** normal arrays

In [43]:
a = [1 2; 3 4]
b_slice = a[:, 2]
@show typeof(b_slice)
@show typeof(a)
@views b = a[:, 2]
@show typeof(b);

typeof(b_slice) = Array{Int64,1}
typeof(a) = Array{Int64,2}
typeof(b) = SubArray{Int64,1,Array{Int64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}


The type of `b` is a good example of how types are not as they may sometimes

Similarly

In [44]:
a = [1 2; 3 4]
b = a' # transpose
typeof(b)

Adjoint{Int64,Array{Int64,2}}

To copy into a typical, dense array

In [45]:
a = [1 2; 3 4]
b = a' # transpose
c = Matrix(b) # convert to matrix
d = collect(b) # also `collect` works on any iterable
c == d

true

## Special Matrices

As we saw with the `transpose`, sometimes types that look like matrices are not stored as a dense array

As an example, consider creating a diagonal matrix

In [46]:
using LinearAlgebra
d = [1.0, 2.0]
a = Diagonal(d)

2×2 Diagonal{Float64,Array{Float64,1}}:
 1.0   ⋅ 
  ⋅   2.0

As you can see, the type is `2×2 Diagonal{Float64,Array{Float64,1}}`, which is not a 2-dimensional array

The reasons for this are both efficiency in storage, as well as efficiency in arithmetic and matrix operations

For example, this both behaves (and is, in an important sense) like any other Matrix

In [47]:
@show 2a
b = rand(2,2)
@show b * a;

2a = [2.0 0.0; 0.0 4.0]
b * a = [0.277454 1.2566; 0.193208 0.276051]


Another example is in the construction of an identity matrix, where a naive implementation is

In [48]:
b = [1.0 2.0; 3.0 4.0]
b - Diagonal([1.0, 1.0]) # poor style, inefficient code

2×2 Array{Float64,2}:
 0.0  2.0
 3.0  3.0

Whereas you should instead use

In [49]:
b = [1.0 2.0; 3.0 4.0]
b - I # good style, and note the lack of dimensions of I

2×2 Array{Float64,2}:
 0.0  2.0
 3.0  3.0

While the implementation of `I` is a little abstract to go into at this point, a hint is that

In [50]:
typeof(I)

UniformScaling{Bool}

So that this is a `UniformScaling` type rather than an identity matrix, making it much more powerful and general

## Assignment and Passing Arrays

As discussed above, in Julia, the left hand side of an assignment is a “binding” to a name

In [51]:
x = [1 2 3]
y = x # name y binds to whatever `x` bound to

1×3 Array{Int64,2}:
 1  2  3

The consequence of this, is that you can re-bind that name

In [52]:
x = [1 2 3]
y = x # name y binds to whatever `x` bound to
z = [2 3 4]
y = z # just changes name binding, not value!
@show (x, y, z);

(x, y, z) = ([1 2 3], [2 3 4], [2 3 4])


What this means is that if `a` is an array and we set `b = a` then `a` and `b` point to exactly the same data

In the above, suppose you had meant to change the value of `x` to the values of `y`, you need to assign the values rather than the name

In [53]:
x = [1 2 3]
y = x # name y binds to whatever `x` bound to
z = [2 3 4]
y .= z # Now dispatches the assignment of each element
@show (x, y, z);

(x, y, z) = ([2 3 4], [2 3 4], [2 3 4])


Alternatively, you could have used `y[:] = z`

This applies to in-place functions as well

First, define a simple function for a linear map

In [54]:
function f(x)
    return [1 2; 3 4] * x # matrix * column vector
end
val = [1, 2]
f(val)

2-element Array{Int64,1}:
  5
 11

In general, these “out-of-place” functions are preferred to “in-place” functions, which modify the arguments

In [55]:
function f(x)
    return [1 2; 3 4] * x # matrix * column vector
end
val = [1, 2]
y = similar(val)
function f!(out, x)
    out .= [1 2; 3 4] * x
end
f!(y, val)
y

2-element Array{Int64,1}:
  5
 11

This demonstrates a key convention in Julia: functions which modify any of the arguments have the name ending with `!` (e.g. `push!`)

We can also see a common mistake, where instead of modifying the arguments, the name binding is swapped

In [56]:
function f(x)
    return [1 2; 3 4] * x # matrix * column vector
end
val = [1, 2]
y = similar(val)

function f!(out, x)
    out = [1 2; 3 4] * x # MISTAKE! should be .= or [:]
end
f!(y, val)
y

2-element Array{Int64,1}:
 140028343950864
 140028343950880

The frequency of making this mistake is one of the reasons to avoid in-place functions, unless proven to be necessary by benchmarking

# Operations on Arrays

## Array Methods

Julia provides standard functions for acting on arrays, some of which we’ve
already seen

In [57]:
a = [-1, 0, 1]


@show length(a)
@show sum(a)
@show mean(a)
@show std(a) #standard deviation
@show var(a) # variance
@show maximum(a)
@show minimum(a)
@show extrema(a) # (mimimum(a), maximum(a))

length(a) = 3
sum(a) = 0
mean(a) = 0.0
std(a) = 1.0
var(a) = 1.0
maximum(a) = 1
minimum(a) = -1
extrema(a) = (-1, 1)


(-1, 1)

To sort an array

In [58]:
b = sort(a, rev = true)  # returns new array, original not modified

3-element Array{Int64,1}:
  1
  0
 -1

In [59]:
b = sort!(a, rev = true)  # returns *modified original* array

3-element Array{Int64,1}:
  1
  0
 -1

In [60]:
b == a  # tests if have the same values

true

In [61]:
b === a  # tests if arrays are identical (i.e share same memory)

true

## Matrix Algebra

For two dimensional arrays, `*` means matrix multiplication

In [62]:
a = ones(1, 2)

1×2 Array{Float64,2}:
 1.0  1.0

In [63]:
b = ones(2, 2)

2×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0

In [64]:
a * b

1×2 Array{Float64,2}:
 2.0  2.0

In [65]:
b * a'

2×1 Array{Float64,2}:
 2.0
 2.0

To solve the linear system `A X = B` for `X` use `A \ B`

In [66]:
A = [1 2; 2 3]

2×2 Array{Int64,2}:
 1  2
 2  3

In [67]:
B = ones(2, 2)

2×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0

In [68]:
A \ B

2×2 Array{Float64,2}:
 -1.0  -1.0
  1.0   1.0

In [69]:
inv(A) * B

2×2 Array{Float64,2}:
 -1.0  -1.0
  1.0   1.0

Although the last two operations give the same result, the first one is numerically more stable and should be preferred in most cases

Multiplying two **one** dimensional vectors gives an error — which is reasonable since the meaning is ambiguous

```julia
ones(2) * ones(2)
```


If you want an inner product in this setting use `dot()` or the `` `unicode ``\dot<TAB>`

In [70]:
dot(ones(2), ones(2))

2.0

Matrix multiplication using one dimensional vectors is a bit inconsistent — pre-multiplication by the matrix is OK, but post-multiplication gives an error

In [71]:
b = ones(2, 2)

2×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0

In [72]:
b * ones(2)

2-element Array{Float64,1}:
 2.0
 2.0

```julia
ones(2) * b
```


## Elementwise Operations

### Algebraic Operations

Suppose that we wish to multiply every element of matrix `A` with the corresponding element of matrix `B`

In that case we need to replace `*` (matrix multiplication) with `.*` (elementwise multiplication)

For example, compare

In [73]:
ones(2, 2) * ones(2, 2)   # Matrix multiplication

2×2 Array{Float64,2}:
 2.0  2.0
 2.0  2.0

In [74]:
ones(2, 2) .* ones(2, 2)   # Element by element multiplication

2×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0

This is a general principle: `.x` means apply operator `x` elementwise

In [75]:
A = -ones(2, 2)

2×2 Array{Float64,2}:
 -1.0  -1.0
 -1.0  -1.0

In [76]:
A.^2  # Square every element

2×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0

However in practice some operations are mathematically valid without broadcasting, and hence the `.` can be omitted

In [77]:
ones(2, 2) + ones(2, 2)  # same as ones(2, 2) .+ ones(2, 2)

2×2 Array{Float64,2}:
 2.0  2.0
 2.0  2.0

Scalar multiplication is similar

In [78]:
A = ones(2, 2)

2×2 Array{Float64,2}:
 1.0  1.0
 1.0  1.0

In [79]:
2 * A  # same as 2 .* A

2×2 Array{Float64,2}:
 2.0  2.0
 2.0  2.0

In fact you can omit the `*` altogether and just write `2A`

Unlike matlab and other languages, scalar addition requires the `.+` in order to correctly broadcast

In [80]:
x = [1, 2]
x .+ 1 # i.e. not x + 1
x .- 1 # i.e. not x - 1

2-element Array{Int64,1}:
 0
 1

### Elementwise Comparisons

Elementwise comparisons also use the `.x` style notation

In [81]:
a = [10, 20, 30]

3-element Array{Int64,1}:
 10
 20
 30

In [82]:
b = [-100, 0, 100]

3-element Array{Int64,1}:
 -100
    0
  100

In [83]:
b .> a

3-element BitArray{1}:
 false
 false
  true

In [84]:
a .== b

3-element BitArray{1}:
 false
 false
 false

We can also do comparisons against scalars with parallel syntax

In [85]:
b

3-element Array{Int64,1}:
 -100
    0
  100

In [86]:
b .> 1

3-element BitArray{1}:
 false
 false
  true

This is particularly useful for *conditional extraction* — extracting the elements of an array that satisfy a condition

In [87]:
a = randn(4)

4-element Array{Float64,1}:
  0.6016463483665031 
  1.1054506620971916 
  0.19547187006475988
 -0.7650089990450518 

In [88]:
a .< 0

4-element BitArray{1}:
 false
 false
 false
  true

In [89]:
a[a .< 0]

1-element Array{Float64,1}:
 -0.7650089990450518

### Changing Dimensions

The primary function for changing the dimension of an array is `reshape()`

In [90]:
a = [10, 20, 30, 40]

4-element Array{Int64,1}:
 10
 20
 30
 40

In [91]:
b = reshape(a, 2, 2)

2×2 Array{Int64,2}:
 10  30
 20  40

In [92]:
b

2×2 Array{Int64,2}:
 10  30
 20  40

Notice that this function returns a “view” on the existing array

This means that changing the data in the new array will modify the data in the
old one:

In [93]:
b[1, 1] = 100  # Continuing the previous example

100

In [94]:
b

2×2 Array{Int64,2}:
 100  30
  20  40

In [95]:
a

4-element Array{Int64,1}:
 100
  20
  30
  40

To collapse an array along one dimension you can use `dropdims()`

In [96]:
a = [1 2 3 4]  # Two dimensional

1×4 Array{Int64,2}:
 1  2  3  4

In [97]:
dropdims(a, dims = 1)

4-element Array{Int64,1}:
 1
 2
 3
 4

The return value is an array with the specified dimension “flattened”

## Broadcasting Functions

Julia provides standard mathematical functions such as `log`, `exp`, `sin`, etc.

In [98]:
log(1.0)

0.0

By default, these functions act *elementwise* on arrays

In [99]:
log.(1:4)

4-element Array{Float64,1}:
 0.0               
 0.6931471805599453
 1.0986122886681098
 1.3862943611198906

Note that we can get the same result as with a comprehension or more explicit loop

In [100]:
[ log(x) for x in 1:4 ]

4-element Array{Float64,1}:
 0.0               
 0.6931471805599453
 1.0986122886681098
 1.3862943611198906

Nonetheless the syntax is convenient

# Introduction to Types

We will discuss this in detail in generic and functional programming, but much of its performance gains and generality of notation comes from Julia’s type system

For example, compare

In [101]:
x = [1, 2, 3]

3-element Array{Int64,1}:
 1
 2
 3

Gives `Array{Int64,1}` as the type whereas

In [102]:
x = [1.0, 2.0, 3.0]

3-element Array{Float64,1}:
 1.0
 2.0
 3.0

These return `Array{Int64,1}` and `Array{Float64,1}` respectively, which the compiler is able to infer from the right hand side of the expressions

Given the information on the type, the compiler can work through the sequence of expressions to infer other types

In [103]:
# define some function
f(y) = 2y

# call with an integer array
x = [1, 2, 3]
z = f(x) # compiler deduces type

3-element Array{Int64,1}:
 2
 4
 6

## Good Practices for Functions and Variables

In order to keep many of the benefits of Julia, you will sometimes want to help the compiler ensure that it can always deduce a single type from any function or expression

As an example of bad practice, is to use an array to hold unrelated types

In [104]:
x = [1.0, "test"] # poor style

2-element Array{Any,1}:
 1.0    
  "test"

The type of this is `Array{Any,1}`, where the `Any` means the compiler has determined that any valid Julia types can be added to the array

While occasionally useful, this is to be avoided whenever possible in performance sensitive code

The other place this can come up is in the declaration of functions,

As an example, consider a function which returns different types depending on the arguments

In [105]:
function f(x)
    if x > 0
        return 1.0
    else
        return 0 # Probably meant 0.0
    end
end
@show f(1)
@show f(-1)

f(1) = 1.0
f(-1) = 0


0

The issue here is relatively subtle:  `1.0` is a floating point, while `0` is an integer

Consequently, given the type of `x`, the compiler cannot in general determine what type the function will return

This issue, called **type stability** is at the heart of most Julia performance considerations

Luckily, the practice of trying to ensure that functions return the same types is also the most consistent with simple, clear code

## Manually Declaring Types

While we keep talking about types, you will notice that we have never declared any types in the underlying code

This is intentional for exposition and “user” code of packages, rather than the writing of those packages themselves

It is also in contrast to some of the sample code you will see

To give an example of the declaration of types, the following are equivalent

In [106]:
function f(x, A)
    b = [5.0; 6.0]
    return A * x .+ b
end
val = f([0.1, 2.0], [1.0 2.0; 3.0 4.0])

2-element Array{Float64,1}:
  9.1
 14.3

In [107]:
function f2(x::Vector{Float64}, A::Matrix{Float64})::Vector{Float64} # argument and return types
    b::Vector{Float64} = [5.0; 6.0]
    return A * x .+ b
end
val = f2([0.1; 2.0], [1.0 2.0; 3.0 4.0])

2-element Array{Float64,1}:
  9.1
 14.3

While declaring the types may be verbose, would it ever generate faster code?

The answer is: almost never

Furthermore, it can lead to confusion and inefficiencies since many things that behave like vectors and matrices are not `Matrix{Float64}` and `Vector{Float64}`

To see a few examples where the first works and the second fails

In [108]:
@show f([0.1; 2.0], [1 2; 3 4])
@show f([0.1; 2.0], Diagonal([1.0, 2.0]))

#f2([0.1; 2.0], [1 2; 3 4]) # not a Float64
#f2([0.1; 2.0], Diagonal([1.0, 2.0])) # not a Matrix{Float64}

f([0.1; 2.0], [1 2; 3 4]) = [9.1, 14.3]
f([0.1; 2.0], Diagonal([1.0, 2.0])) = [5.1, 10.0]


2-element Array{Float64,1}:
  5.1
 10.0

# Linear Algebra

Julia provides some a great deal of additional functionality related to linear operations

In [109]:
A = [1 2; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [110]:
det(A)

-2.0

In [111]:
tr(A)

5

In [112]:
eigvals(A)

2-element Array{Float64,1}:
 -0.3722813232690143
  5.372281323269014 

In [113]:
rank(A)

2

For more details see the [linear algebra section](https://docs.julialang.org/en/stable/manual/linear-algebra/) of the standard library

# Ranges

As with many other types, a `Range` can act as a vector

In [114]:
a = 10:12 # a range, equivalent to 10:1:12
@show Vector(a) # can convert, but shouldn't

b = Diagonal([1.0, 2.0, 3.0])
b * a .- [1.0; 2.0; 3.0]

Vector(a) = [10, 11, 12]


3-element Array{Float64,1}:
  9.0
 20.0
 33.0

Ranges can also be created with floating point numbers using the same notation

In [115]:
a = 0.0:0.1:1.0 # 0.0, 0.1, 0.2, ... 1.0

0.0:0.1:1.0

But care should be taken if the terminal node is not a multiple of the set sizes

In [129]:
maxval = 1.0
minval = 0.0
stepsize = 0.15
a = minval:stepsize:maxval # 0.0, 0.15, 0.3, ... ??? Not 1.0
@show a
maximum(a) == maxval

a = 0.0:0.15:0.9


false

To evenly space points where the maximum value is important, i.e., `linspace` in other languages

In [130]:
maxval = 1.0
minval = 0.0
numpoints = 10
a = range(minval, stop=maxval, length=numpoints)
@show a
# a = range(minval, maxval, length=numpoints) # supported soon...
maximum(a) == maxval

a = 0.0:0.1111111111111111:1.0


true

# Tuples and Named Tuples

We were introduced to Tuples earlier, which provide high-performance but immutable sets of distinct types

In [118]:
t = (1.0, "test")
t[1] # access by index
a, b = t # unpack
# t[1] = 3.0 # would fail as tuples are immutable
println("a = $a and b = $b")

a = 1.0 and b = test


As well as named tuples, which are tuples extended to have a list of names

In [119]:
t = (val1 = 1.0, val2 = "test")
t.val1 # access by index
a, b = t # unpack
println("a = $a and b = $b") # access by index

a = 1.0 and b = test


While immutable, it is possible to manipulate tuples and generate new ones

In [120]:
t2 = (val3 = 4, val4 = "test!!")
t3 = merge(t, t2) # new tuple

(val1 = 1.0, val2 = "test", val3 = 4, val4 = "test!!")

Named tuples are a convenient and high-performance way to manage and unpack sets of parameters

In [121]:
function f(parameters)
    α, β = parameters.α, parameters.β # poor style, error prone if adding parameters
    return α + β
end
paramvals = (α = 0.1, β = 0.2)
f(paramvals)

0.30000000000000004

This functionality is aided by the `Parameters.jl` package and the @unpack macro

In [131]:
using Parameters
function f(parameters)
    @unpack α, β = parameters # good style, less sensitive to errors
    return α + β
end
parameters = (α = 0.1, β = 0.2)
f(parameters)

0.30000000000000004

In order to manage default values, use the `@with_kw` macro

In [123]:
using Parameters
paramgen = @with_kw (α = 0.1, β = 0.2) # creates named tuples with defaults

# creates named tuples, replacing defaults
@show paramgen()
@show paramgen(α = 0.2)
@show paramgen(α = 0.2, β = 0.5);

paramgen() = (α = 0.1, β = 0.2)
paramgen(α=0.2) = (α = 0.2, β = 0.2)
paramgen(α=0.2, β=0.5) = (α = 0.2, β = 0.5)


# Exercises


<a id='np-ex1'></a>

## Exercise 1

This exercise is on some matrix operations that arise in certain problems, including when dealing with linear stochastic difference equations

If you aren’t familiar with all the terminology don’t be concerned — you can skim read the background discussion and focus purely on the matrix exercise

With that said, consider the stochastic difference equation


<a id='equation-ja-sde'></a>
<table width=100%><tr style='background-color: #FFFFFF !important;'>
<td width=10%></td>
<td width=80%>
$$
X_{t+1} = A X_t + b + \Sigma W_{t+1}
$$
</td><td width=10% style='text-align:center !important;'>
(1)
</td></tr></table>

Here

- $ X_t, b $ and $ X_{t+1} $ ar $ n \times 1 $  
- $ A $ is $ n \times n $  
- $ \Sigma $ is $ n \times k $  
- $ W_t $ is $ k \times 1 $ and $ \{W_t\} $ is iid with zero mean and variance-covariance matrix equal to the identity matrix  


Let $ S_t $ denote the $ n \times n $ variance-covariance matrix of $ X_t $

Using the rules for computing variances in matrix expressions, it can be shown from [(1)](#equation-ja-sde) that $ \{S_t\} $ obeys


<a id='equation-ja-sde-v'></a>
<table width=100%><tr style='background-color: #FFFFFF !important;'>
<td width=10%></td>
<td width=80%>
$$
S_{t+1} = A S_t A' + \Sigma \Sigma'
$$
</td><td width=10% style='text-align:center !important;'>
(2)
</td></tr></table>

It can be shown that, provided all eigenvalues of $ A $ lie within the unit circle, the sequence $ \{S_t\} $ converges to a unique limit $ S $

This is the **unconditional variance** or **asymptotic variance** of the stochastic difference equation

As an exercise, try writing a simple function that solves for the limit $ S $ by iterating on [(2)](#equation-ja-sde-v) given $ A $ and $ \Sigma $

To test your solution, observe that the limit $ S $ is a solution to the matrix equation


<a id='equation-ja-dle'></a>
<table width=100%><tr style='background-color: #FFFFFF !important;'>
<td width=10%></td>
<td width=80%>
$$
S = A S A' + Q
\quad \text{where} \quad Q := \Sigma \Sigma'
$$
</td><td width=10% style='text-align:center !important;'>
(3)
</td></tr></table>

This kind of equation is known as a **discrete time Lyapunov equation**

The [QuantEcon package](http://quantecon.org/julia_index.html)
provides a function called `solve_discrete_lyapunov` that implements a fast
“doubling” algorithm to solve this equation

Test your iterative method against `solve_discrete_lyapunov` using matrices

$$
A =
\begin{bmatrix}
    0.8 & -0.2  \\
    -0.1 & 0.7
\end{bmatrix}
\qquad
\Sigma =
\begin{bmatrix}
    0.5 & 0.4 \\
    0.4 & 0.6
\end{bmatrix}
$$

# Solutions

## Exercise 1

Here’s the iterative approach

In [124]:
function compute_asymptotic_var(A, Sigma;
                                S0 = Sigma * Sigma',
                                tolerance = 1e-6,
                                maxiter = 500)
    V = Sigma * Sigma'
    S = S0
    err = tolerance + 1
    i = 1
    while err > tolerance && i ≤ maxiter
        next_S = A * S * A' + V
        err = norm(S - next_S)
        S = next_S
        i += 1
    end
    return S
end

compute_asymptotic_var (generic function with 1 method)

In [125]:
A =     [0.8 -0.2;
        -0.1 0.7]
Sigma = [0.5 0.4;
         0.4 0.6]

2×2 Array{Float64,2}:
 0.5  0.4
 0.4  0.6

Note that all eigenvalues of $ A $ lie inside the unit disc:

In [126]:
maximum(abs, eigvals(A))

0.9

Let’s compute the asymptotic variance:

In [127]:
our_solution = compute_asymptotic_var(A, Sigma)

2×2 Array{Float64,2}:
 0.671228  0.633476
 0.633476  0.858874

Now let’s do the same thing using QuantEcon’s solve_discrete_lyapunov() function and check we get the same result

In [128]:
using QuantEcon
norm(our_solution - solve_discrete_lyapunov(A, Sigma * Sigma'))

3.883245447999784e-6